# Object Oriented Image Processing - 2018/05/25

jeremy.moore@txstate.edu

In [1]:
current_project = 'BOR-Reports_1929-1930'  # RENAME TITLE OF JUPYTER NOTEBOOK TO MATCH

In [4]:
## Set all options (Import, Directories, Options)

# make Jupyter Notebook take up the ENTIRE WIDTH
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# matplotlib options
# magic that lets us plot directly in the notebook
# can switch from notebook to inline, but MUST restart kernel to switch back to notebook
# https://stackoverflow.com/questions/41125690/matplotlib-notebook-showing-a-blank-histogram
%matplotlib qt

# === IMPORT ===
# built-in
import logging
from math import isnan
from pathlib import Path
from subprocess import call, check_output

# 3rd party
import cv2
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

# jeremy's
import img_qc.img_qc as img_qc

# === DIRECTORIES ===

# set current directory
current_directory = Path.cwd()

# in case we get fancy and change our current directory later
start_directory = current_directory

# set home directory
home_directory = Path.home()

bor_reports = Path('/Users/Jeremy/PycharmProjects/BOR-Reports_autocrop/')

# set quality control directory as hard-coded: Path.cwd()/data/demo-images_no-crop/{current_project}
qc_directory = bor_reports.joinpath(current_project)

# === OPTIONS ===

# on a high-dpi monitor this will increase the quality of plots on-screen
%config InlineBackend.figure_format = 'retina'

# Pandas options
pd.set_option('display.max_colwidth', -1)  # won't truncate output such as Path names

# Logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logging.info(f'qc_directory: {qc_directory}')

# set Image Metadata CSV information

# set CSV filename
csv_name = current_project + '.csv'

# set CSV path
csv_path = qc_directory.joinpath(csv_name)

# === LOAD IMAGE METADATA ===
if csv_path.is_file():
    logging.info(f'{csv_path} exists')
    logging.info(f'loading metadata from {csv_name}')
    
    # create DataFrame from CSV
    images_df = pd.read_csv(csv_path)

else:
    logging.info(f'{csv_name} does NOT exist')
    logging.info(f'loading metdata from {qc_directory}')
    
    # create DataFrame with Exiftools
    images_df = img_qc.get_images_df(str(qc_directory), 'jpg')
    
    # write metadata to CSV file
    images_df.to_csv(csv_path)
    if csv_path.is_file():
        print(f'{csv_name} NOW EXISTS')

2018-05-26 16:24:12,530 - root - INFO - qc_directory: /Users/Jeremy/PycharmProjects/BOR-Reports_autocrop/BOR-Reports_1929-1930
2018-05-26 16:24:12,532 - root - INFO - /Users/Jeremy/PycharmProjects/BOR-Reports_autocrop/BOR-Reports_1929-1930/BOR-Reports_1929-1930.csv exists
2018-05-26 16:24:12,533 - root - INFO - loading metadata from BOR-Reports_1929-1930.csv


In [ ]:
# create DigitalObject from qc_directory
qc = img_qc.DigitalObject(qc_directory)

my = img_qc.DigitalImage(qc.jpg_list[8])

print(my.path, my.height, my.width, my.stem, my.extension)

img_qc.show_cv2_image(my.cv2_image)

## Compare thresholding & display speed: Pillow vs. OpenCV

In [ ]:
%%timeit

bitonal = img_qc.get_pil_bitonal(my.pil_image, 125)

plt.imshow(bitonal)

In [ ]:
%%timeit
bitonal = img_qc.get_cv2_bitonal(my.cv2_image, 125)

img_qc.show_cv2_image(bitonal, my.path)

In [ ]:
def process_image(image_path,
                  autocrop_height,
                  qc_height,
                  padding,
                  deskew_padding,
                  size_x,
                  size_y,
                  location_x,
                  location_x_pixel,
                  location_y,
                  location_y_pixel,
                  rotate_image,
                  rotate_image_float
                 ):
    global crop_box, deskew_box, deskew_angle
    my = img_qc.DigitalImage(image_path)

    # load image
    image = my.cv2_image
    
    # rotate image
    image = img_qc.rotate_bound(image, rotate_image)
    
    # get auto-crop bounding box
    rectangle, theta, (x, y) = img_qc.get_autocrop_bounding_box(autocrop_height, image)
    
    # get the crop and deskew boxes
    crop_box, deskew_box = img_qc.get_crop_box(rectangle,
                                               padding,
                                               deskew_padding,
                                               size_x,
                                               size_y,
                                               location_x,
                                               location_x_pixel,
                                               location_y,
                                               location_y_pixel
                                              )
    
    # get deskew angle
    deskew_angle = img_qc.get_deskew_angle(image, deskew_box)
    
    # add tenths to theta (by subtracting for correct angle)
    theta -= rotate_image_float
    
    # draw crop and deskew_boxes
    image = img_qc.draw_crop_box(qc_height, crop_box, deskew_box, x, y, theta, image)
    
    # show the image
    img_qc.show_cv2_image(image)
    



In [ ]:
# === AutoCrop Widget

# == AutoCrop Height Slider
autocrop_height_slider = widgets.IntSlider(description='crop height', min=100, max=1000, step=100, value=500, continuous_update=False)

# QC Height Slider
qc_height_slider = widgets.IntSlider(description='qc height', min=50, max=1000, step=50, value=750, continuous_update=False)

# Padding Slider
padding_slider = widgets.IntSlider(description='padding', min=-500, max=500, step=10, value=50, continuous_update=False)

# == X Sliders
location_x_slider = widgets.IntSlider(description='x +/- 10px', min=-500, max=500, step=10, value=0, continuous_update=False)
location_x_pixel_slider = widgets.IntSlider(description='x +/- 1px', min=-10, max=10, step=1, value=0, continuous_update=False)
size_x_slider = widgets.IntSlider(description='size x', min=-500, max=500, step=10, value=0, continuous_update=False)

# == Y Sliders
location_y_slider = widgets.IntSlider(description='y +/- 10px', min=-500, max=500, step=10, value=0, continuous_update=False)
location_y_pixel_slider = widgets.IntSlider(description='y +/- 1px', min=-10, max=10, step=1, value=0, continuous_update=False)
size_y_slider = widgets.IntSlider(description='size y', min=-500, max=500, step=10, value=0, continuous_update=False)

# == Image Path Dropdown
image_path_options = qc.jpg_list
image_path_dropdown = widgets.Dropdown(options=image_path_options, value=image_path_options[0])

# == Rotate Image Sliders
rotate_image_slider = widgets.IntSlider(description='rotate', min=-90, max=270, step=90, value=0, continuous_update=False)
rotate_image_float_slider = widgets.FloatSlider(description='rotate 0.01', min=-5, max=5, step=0.01, value=0, continuous_update=False)

# == Threshold Value Slider
threshold_value_slider = widgets.IntSlider(description='threshold', min=15, max=245, value=127, continuous_update=False)

# == Verify Image Saved Valid/Invalid
verify_widget = widgets.Valid(value=False, description='Saved?')

# == Save Button
save_button = widgets.Button(value=False, description='Save Crop', button_style='warning')

# == Advance Radio Buttons
advance_radio_buttons = widgets.RadioButtons(description='On Save:', options=[0, 1, 2], value=1)

# Image Type Toggle Buttons
image_type_button = widgets.ToggleButtons(options=['RGB', 'Bitonal'], value='RGB')

# AutoCrop Radio Buttons
autocrop_button = widgets.Button(description='AutoCropArr', button_style='danger')

def on_click_autocrop_button(click):
    print('AutoCrop')
    
autocrop_button.on_click(on_click_autocrop_button)

# Deskew Button
deskew_button = widgets.Button(description='DeskewArr', button_style='success')

def on_click_deskew_button(click):
    rotate_image_float_slider.value = -deskew_angle
    print(deskew_angle)
deskew_button.on_click(on_click_deskew_button)

# == Deskew Padding Slider
deskew_padding_slider = widgets.IntSlider(description='deskew padding', min=-500, max=500, step=10, value=-250, continuous_update=False)

row1 = widgets.HBox([image_path_dropdown, autocrop_height_slider, qc_height_slider])
row2 = widgets.HBox([location_x_slider, location_y_slider, rotate_image_slider])
row3 = widgets.HBox([location_x_pixel_slider, location_y_pixel_slider, rotate_image_float_slider])
row4 = widgets.HBox([size_x_slider, size_y_slider, deskew_padding_slider])
row5 = widgets.HBox([threshold_value_slider, padding_slider])
row6 = widgets.HBox([image_type_button, autocrop_button, deskew_button])
row7 = widgets.HBox([save_button, advance_radio_buttons, verify_widget])
ui = widgets.VBox([row1, row2, row3, row4, row5, row6, row7])

In [ ]:
out = widgets.interactive_output(process_image, {'image_path': image_path_dropdown,
                                                 'autocrop_height': autocrop_height_slider,
                                                 'qc_height': qc_height_slider,
                                                 'padding': padding_slider,
                                                 'deskew_padding': deskew_padding_slider,
                                                 'size_x': size_x_slider,
                                                 'size_y': size_y_slider,
                                                 'location_x': location_x_slider,
                                                 'location_x_pixel': location_x_pixel_slider,
                                                 'location_y': location_y_slider, 
                                                 'location_y_pixel': location_y_pixel_slider,
                                                 'rotate_image': rotate_image_slider,
                                                 'rotate_image_float': rotate_image_float_slider
                                                })

display(ui, out)

In [ ]:
rectangle, theta, (x, y) = img_qc.get_autocrop_bounding_box(500, my.path)
image = img_qc.get_cv2_image(my.path)
crop_box, deskew_box = img_qc.get_crop_box(rectangle,
                                    padding=-50,
                                    deskew_padding=-150,
                                    size_x=0,
                                    size_y=0,
                                    location_x=-100,
                                    location_x_pixel=0,
                                    location_y=0,
                                    location_y_pixel=0
                                   )
deskew_angle = img_qc.get_deskew_angle(image, deskew_box)
crop_box_image = img_qc.draw_crop_box(750, crop_box, deskew_box, x, y, theta, my.path)

img_qc.show_cv2_image(crop_box_image, my.path)

In [ ]:
crop_box_image = img_qc.draw_crop_box(750, crop_box, deskew_box, x, y, deskew_angle, my.path)
print(deskew_angle)

img_qc.show_cv2_image(crop_box_image, my.path)

In [ ]:
# parameters for matplotlib to increase our default figure size -- NOTE: figure sizes are in INCHES
plt.rcParams["figure.figsize"] = (6, 8)  # set as needed for your screen and eyes

In [ ]:
# create DigitalObject from qc_directory
qc = DigitalObject(qc_directory)

# get first item in JPEG list
print(qc.jpg_list[0])

image_path_options = qc.jpg_list

In [ ]:
# === AutoCrop Widget

# == AutoCrop Height Slider
autocrop_height_slider = widgets.IntSlider(description='crop height', min=100, max=1000, step=100, value=500, continuous_update=False)

# QC Height Slider
qc_height_slider = widgets.IntSlider(description='qc height', min=50, max=1000, step=50, value=600, continuous_update=False)

# Padding Slider
padding_slider = widgets.IntSlider(description='padding', min=-500, max=500, step=10, value=50, continuous_update=False)

# == X Sliders
location_x_slider = widgets.IntSlider(description='x +/- 10px', min=-500, max=500, step=10, value=0, continuous_update=False)
location_x_pixel_slider = widgets.IntSlider(description='x +/- 1px', min=-10, max=10, step=1, value=0, continuous_update=False)
size_x_slider = widgets.IntSlider(description='size x', min=-500, max=500, step=10, value=0, continuous_update=False)

# == Y Sliders
location_y_slider = widgets.IntSlider(description='y +/- 10px', min=-500, max=500, step=10, value=0, continuous_update=False)
location_y_pixel_slider = widgets.IntSlider(description='y +/- 1px', min=-10, max=10, step=1, value=0, continuous_update=False)
size_y_slider = widgets.IntSlider(description='size y', min=-500, max=500, step=10, value=0, continuous_update=False)

# == Image Path Dropdown
image_path_dropdown = widgets.Dropdown(options=image_path_options, value=image_path_options[0])

# == Rotate Image Sliders
rotate_image_slider = widgets.IntSlider(description='rotate', min=-90, max=270, step=90, value=0, continuous_update=False)
rotate_image_tenths_slider = widgets.FloatSlider(description='rotate 0.1', min=-5, max=5, step=0.01, value=0, continuous_update=False)

# == Threshold Value Slider
threshold_value_slider = widgets.IntSlider(description='threshold', min=15, max=245, value=127, continuous_update=False)

# == Verify Image Saved Valid/Invalid
verify_widget = widgets.Valid(value=False, description='Saved?')

# == Save Button
save_button = widgets.Button(value=False, description='Save Crop', button_style='warning')

# == Advance Radio Buttons
advance_radio_buttons = widgets.RadioButtons(description='On Save:', options=[0, 1, 2], value=1)

# Image Type Toggle Buttons
image_type_button = widgets.ToggleButtons(options=['RGB', 'Bitonal'], value='RGB')

# AutoCrop Button
autocrop_button = widgets.Button(description='AutoCrop')

# Deskew Button
deskew_button = widgets.Button(description='Deskew Text')

# == Deskew Padding Slider
deskew_padding_slider = widgets.IntSlider(description='deskew padding', min=-500, max=500, step=10, value=-250, continuous_update=False)

row1 = widgets.HBox([image_path_dropdown, autocrop_height_slider, qc_height_slider])
row2 = widgets.HBox([location_x_slider, location_y_slider, rotate_image_slider])
row3 = widgets.HBox([location_x_pixel_slider, location_y_pixel_slider, rotate_image_tenths_slider])
row4 = widgets.HBox([size_x_slider, size_y_slider, deskew_padding_slider])
row5 = widgets.HBox([threshold_value_slider, padding_slider])
row6 = widgets.HBox([image_type_button, autocrop_button, deskew_button])
row7 = widgets.HBox([save_button, advance_radio_buttons, verify_widget])
ui = widgets.VBox([row1, row2, row3, row4, row5, row6, row7])

out = widgets.interactive_output(get_crop_contour, {'autocrop_height': autocrop_height_slider,
                                                    'qc_height': qc_height_slider,
                                                    'padding': padding_slider,
                                                    'location_x': location_x_slider,
                                                    'location_x_pixel': location_x_pixel_slider,
                                                    'size_x': size_x_slider,
                                                    'location_y': location_y_slider,
                                                    'location_y_pixel': location_y_pixel_slider,
                                                    'size_y': size_y_slider,
                                                    'deskew_padding': deskew_padding_slider,
                                                    'rotate_image': rotate_image_slider,
                                                    'rotate_image_tenths': rotate_image_tenths_slider,
                                                    'threshold_value': threshold_value_slider,
                                                    'image_type': image_type_button,
                                                    'image_path' : image_path_dropdown
                                                   })
#out.layout.height = '850px'

display(ui, out)

def on_dropdown_selected(image_path):
    image_path = Path(image_path_dropdown.value)
    #print(f'image_path: {image_path}')
    cropped_directory = image_path.parent.joinpath('00_AutoCropper')
    
    # set cropped image name
    cropped_image_name = str(image_path.stem) + '.tif'
    
    # set cropped image path
    cropped_image_path = cropped_directory.joinpath(cropped_image_name)
    
    #print(f'cropped path: {cropped_image_path}')
    if cropped_image_path.is_file():
        #print('TRUE')
        verify_widget.value = True
    else:
        #print('FALSE')
        verify_widget.value = False
        
def on_image_type_button_clicked(b):
    print(image_type_button.value)
    image_type = image_type_button.value
    
def on_save_button_clicked(b):
    global image_path_dropdown
    get_cropped_image(autocrop_height=autocrop_height_slider.value,
                      qc_height=qc_height_slider.value,
                      padding=padding_slider.value,
                      location_x=location_x_slider.value,
                      size_x=size_x_slider.value,
                      location_y=location_y_slider.value,
                      size_y=size_y_slider.value,
                      rotate_image=rotate_image_slider.value,
                      rotate_image_tenths=rotate_image_tenths_slider.value,
                      threshold_value=threshold_value_slider.value,
                      image_type=image_type_button.value,
                      image_path=image_path_dropdown.value
                     )
    image_path = Path(image_path_dropdown.value)
    if advance_radio_buttons.value > 0:
        next_image_path = image_path
        next_image_path_stem = next_image_path.stem
        image_index_number = next_image_path_stem[-4:]
        image_index_number = int(image_index_number)
        image_index_number += int(advance_radio_buttons.value)
        if image_index_number < len(image_path_options) or image_index_number == len(image_path_options):
            next_image_path_name = next_image_path_stem[:-4] + str(image_index_number).zfill(4) + next_image_path.suffix
            next_image_path = next_image_path.parent.joinpath(next_image_path_name)
            print(f'next path name: {next_image_path}')
            image_path = next_image_path
            print(f'image1: {image_path}')
        else:
            print('All images cropped')

    image_path_dropdown.value = image_path
    
    cropped_directory = image_path.parent.joinpath('00_AutoCropper')
    
    # set cropped image name
    cropped_image_name = str(image_path.stem + '.tif')
    # set cropped image path
    cropped_image_path = cropped_directory.joinpath(cropped_image_name)
    
    #print(f'cropped path: {cropped_image_path}')
    if cropped_image_path.is_file():
        #print('TRUE')
        verify_widget.value = True
    else:
        #print('FALSE')
        verify_widget.value = False
        
def on_autocrop_button_clicked(b):
    print('AutoCrop!')
    get_crop_contour()

def on_deskew_button_clicked(b):
    print('Deskew!')

save_button.on_click(on_save_button_clicked)
autocrop_button.on_click(on_autocrop_button_clicked)
deskew_button.on_click(on_deskew_button_clicked)
image_path_dropdown.observe(on_dropdown_selected)

In [ ]:
image_test_path = qc.jpg_list[5]
image_test_path

In [ ]:
image_test = cv2.imread(str(image_test_path))
image_test_crop = image_test[400:3800, 50:2200]
show_cv2_image(image_test_crop, image_test_path)

In [ ]:
resized = img_qc.get_resized_cv_image(image_test_crop, height=500)
blurred = cv2.GaussianBlur(resized, (5, 5), 0)
gray = cv2.bitwise_not(blurred)
otsu_value, thresh = get_otsu_threshold_image_and_value(gray)
show_cv2_image(thresh, image_test_path)

In [ ]:
# https://www.pyimagesearch.com/2017/02/20/text-skew-correction-opencv-python/

# grab the (x, y) coordinates of all pixel values that
# are greater than zero, then use these coordinates to
# compute a rotated bounding box that contains all
# coordinates
coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]
 
# the `cv2.minAreaRect` function returns values in the
# range [-90, 0); as the rectangle rotates clockwise the
# returned angle trends to 0 -- in this special case we
# need to add 90 degrees to the angle
if angle < -45:
	angle = -(90 + angle)
 
# otherwise, just take the inverse of the angle to make
# it positive
else:
	angle = -angle

In [ ]:
# rotate the image to deskew it
(h, w) = image_test_crop.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(image_test_crop, M, (w, h),
	flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

In [ ]:
show_cv2_image(rotated, image_test_path)